In [1]:
import pandas as pd

# Data

# http://www.football-data.co.uk/brazil.php Data

## Data Description

### Notes for Football Data

All data is in csv format, ready for use within standard spreadsheet applications. Please note that some abbreviations are no longer in use (in particular odds from specific bookmakers no longer used) and refer to data collected in earlier seasons. For a current list of what bookmakers are included in the dataset please visit http://www.football-data.co.uk/matches.php

#### Key to results data:

- Date = Match Date (dd/mm/yy)
- Home = Home Team
- Away = Away Team
- HG = Full Time Home Team Goals
- AG = Full Time Away Team Goals
- Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)

##### Key to 1X2 (match) betting odds data:

- PH = Pinnacle home win odds
- PD = Pinnacle draw odds
- PA = Pinnacle away win odds

- MaxH = Oddsportal maximum home win odds
- MaxD = Oddsportal maximum draw win odds
- MaxA = Oddsportal maximum away win odds

- AvgH = Oddsportal average home win odds
- AvgD = Oddsportal average draw win odds
- AvgA = Oddsportal average away win odds


Current results (full time, half time)
Xcores - http://www.xcores.com

Bookmakers betting odds
Betbrain - http://www.betbrain.com
Oddsportal - http://www.oddsportal.com
Individual bookmakers

Betting odds for weekend games are collected Friday afternoons, and on Tuesday afternoons for midweek games.


In [2]:
df = pd.read_csv('../Data/Brazil_Serie_A/BRA.csv')

.

In [3]:
df.tail()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
2471,Brazil,Serie A,2018,23/08/18,01:00,Sport Recife,America MG,0.0,2.0,A,1.94,3.26,4.72,2.01,3.50,4.75,1.92,3.23,4.27
2472,Brazil,Serie A,2018,23/08/18,01:45,Fluminense,Corinthians,1.0,0.0,H,2.16,3.14,4.02,2.33,3.22,4.15,2.10,3.06,3.83
2473,Brazil,Serie A,2018,23/08/18,01:45,Gremio,Cruzeiro,1.0,1.0,D,1.88,3.10,5.51,1.94,3.20,5.51,1.86,3.08,4.89
2474,Brazil,Serie A,2018,23/08/18,23:30,Flamengo RJ,Vitoria,1.0,0.0,H,1.28,5.57,13.69,1.36,5.65,13.69,1.29,5.19,10.83
2475,Brazil,Serie A,2018,24/08/18,00:00,Atletico-MG,Vasco,0.0,0.0,D,1.48,4.47,7.57,1.50,4.54,8.50,1.46,4.26,6.93


.

# Scraper

In [4]:
# Import the scraper libraries

In [5]:
from bs4 import BeautifulSoup
import requests
import os, os.path, csv

In [6]:
def date_fixer(date):
    #Remove white spaces
    date = date.strip()
    #Get the last five char
    date = date[-5:]
    #Add the year
    date = date + '/18'
    return date

In [65]:
def clean_html(html):
    # Take the second table wich contains all the info
    html = html[1]
    # Drop the useless columns
    html.drop(labels=[3,4,5], axis=1, inplace=True)
    # Drop the nan values from the third column
    html.dropna(subset=[2], inplace=True)
    return html

In [7]:
# Pinnacle

In [8]:
pinnacle_url = 'https://www.pinnacle.com/en/odds/match/soccer/brazil/brazil-serie-a'

In [9]:
from selenium import webdriver

# start Chrome
options = webdriver.ChromeOptions()
options.add_argument('headless')
try:
    browser = webdriver.Chrome(chrome_options=options)
except:
    pass

browser.get(pinnacle_url)
html = browser.page_source

In [10]:
soup = BeautifulSoup(html, "html.parser")

Date
Time
Home
Away
- PH = Pinnacle home win odds
- PD = Pinnacle draw odds
- PA = Pinnacle away win odds

bindings = soup.find_all('span', class_="ng-binding")

In [11]:
# Scrap date with BS
date_raw = soup.find('span', class_="date-highlight ng-binding ng-scope").text
date = date_fixer(date_raw)

In [63]:
# Scrap:
# - Time
# - Home
# - Away
# - Home Odds
# - Draw Odds
# - Away Odds

df_html = pd.read_html(html)
info_table = clean_html(df_html)

In [64]:
info_table

,0,1,2
0,15.0,Vasco da Gama,2.050
1,NaN,Chapecoense,4.109
2,NaN,Draw,3.320
4,12.0,SC Internacional Porto Alegre,2.190
5,NaN,Palmeiras,3.680
6,NaN,Draw,3.270
8,12.0,America Mineiro,3.570
9,NaN,Flamengo,2.320
10,NaN,Draw,3.110
12,12.0,Vitoria BA,3.120


In [83]:
info_table.loc[0]

0               15
1    Vasco da Gama
2             2.05
Name: 0, dtype: object

In [85]:
notnan(df.iloc[1,0])

NameError: name 'notnan' is not defined

In [105]:
for index, row in info_table.iterrows():
    if(pd.notnull(row[0])):
        
        print(row)

0               15
1    Vasco da Gama
2             2.05
Name: 0, dtype: object
0                               12
1    SC Internacional Porto Alegre
2                             2.19
Name: 4, dtype: object
0                 12
1    America Mineiro
2               3.57
Name: 8, dtype: object
0            12
1    Vitoria BA
2          3.12
Name: 12, dtype: object


In [66]:
df.tail(1)

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
2475,Brazil,Serie A,2018,24/08/18,00:00,Atletico-MG,Vasco,0.0,0.0,D,1.48,4.47,7.57,1.5,4.54,8.5,1.46,4.26,6.93


In [ ]:
# Create an empty DataFrame with the main df headers
df_predict = pd.DataFrame(data=None, columns=df.columns)


new_row = pd.Series([country, league, season, date],
                    index = ['Country', 'League', 'Season',
                             'Date', 'Time', 'Home', 'Away', 
                             'HG','AG', 'Res', 'PH', 'PD', 
                             'PA', 'MaxH', 'MaxD', 'MaxA', 
                             'AvgH', 'AvgD','AvgA'])

country = 'Brazil'
league = 'Serie A'
season = '2018'
date = date
    

In [77]:
# Create an empty DataFrame with the main df headers
df_predict = pd.DataFrame(data=None, columns=df.columns)

In [75]:
def create_row(main_df, date, pinnacle_table):
    

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA


In [78]:

time = 


In [ ]:
row = pd.Series(['time_stamp', ad_id, title, category_id, price, initial_quantity, available_quantity, sold_quantity,
                     status, date_created, var_id, variations, var_name, var_price, var_available_quantity, var_sold_quantity, total_views],
                    index = ['date', 'ad_id', 'title', 'category_id', 'price', 'initial_quantity','available_quantity', 'sold_quantity', 
                             'status', 'date_created', 'var_id','variations','var_name', 'var_price', 'var_available_quantity','var_sold_quantity','total_views'])
            df = df.append(row, ignore_index=True)